# SmoothQuant for qwen
这个东西结构代码和llama一致，所以在smoothquant那里我小改了一下，把作用于llama的函数直接用于qwen看看效果
在最开始需要用校准数据集得到s，这里按照其代码，使用了Pile数据集，理论上可以换，但我觉得不能和之后测试的数据集一样。代码参考原仓库“examples/generate_act_scales.py”

In [1]:

import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant import quantize_model
import tqdm
from datasets import load_dataset

/home/linchi/miniforge3/envs/huggingface/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


这是PPL的评估器

In [2]:
class Evaluator:
    def __init__(self, dataset, tokenizer, device, n_samples=40):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device

        self.dataset = tokenizer(
            "\n\n".join(dataset["text"]), return_tensors="pt"
        ).input_ids.to(device)

        self.n_samples = n_samples

    @torch.no_grad()
    def evaluate(self, model):
        model.eval()
        nlls = []
        n_samples = self.n_samples if self.n_samples else self.dataset.size(1) // 2048
        for i in tqdm.tqdm(range(n_samples), desc="Evaluating..."):
            batch = self.dataset[:, (i * 2048) : ((i + 1) * 2048)].to(model.device)
            with torch.no_grad():
                lm_logits = model(batch).logits
            shift_logits = lm_logits[:, :-1, :].contiguous().float()
            shift_labels = self.dataset[:, (i * 2048) : ((i + 1) * 2048)][:, 1:]
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1)
            )
            neg_log_likelihood = loss.float() * 2048
            nlls.append(neg_log_likelihood)

        return torch.exp(torch.stack(nlls).sum() / (n_samples * 2048))

先加载bf16模型，评估一下，再量化，再评估一下。

In [3]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
act_scales_path = "act_scales/Qwen2.5-0.5B.pt"
alpha=0.85
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
evaluator = Evaluator(dataset, tokenizer, "cuda")

model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.bfloat16, device_map="auto"
)


Token indices sequence length is longer than the specified maximum sequence length for this model (299078 > 131072). Running this sequence through the model will result in indexing errors


先对原模型进行测试

In [ ]:
# ppl = evaluator.evaluate(model)
# print(f"Perplexity: {ppl}")

量化

In [5]:
act_scales = torch.load(act_scales_path)
smooth_lm(model, act_scales, alpha)#smooth之后再量化
model = quantize_model(
        model,
        weight_quant="per_channel",
        act_quant="per_token",
        quantize_bmm_input=True,
    )

量化后测试

In [6]:
ppl = evaluator.evaluate(model)
print(f"Perplexity: {ppl}")

Evaluating...: 100%|██████████| 40/40 [01:35<00:00,  2.38s/it]


Perplexity: 14.359110832214355
